In [57]:
import pandas as pd

- 불러오기

In [58]:
df = pd.read_csv('../../2020.csv',encoding='euc-kr')
df.drop(columns=['등록일자'],inplace=True)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200101,1호선,종각,20427,16301
1,20200101,1호선,시청,12126,10516
2,20200101,우이신설선,신설동,892,828
3,20200101,우이신설선,보문,917,855
4,20200101,우이신설선,성신여대입구(돈암),2010,2363


- 평일 구하기

In [59]:
cols = list(df.columns)[:3]
target = list(df.columns)[3:]
df_list = []
# 반복문에 해당되지 않는 범위는 수동으로 구해서 
for i in range(6, 28, 7): # 사용할때 여기 수정 해당 월에 맞게   여긴 1월2일이 월요일이라 2일 부터 시작 30,31일 데이터는 추가로 더해야함
    start_date = 20200100 + i   # 여기도 조건에 맞게 수정
    end_date = start_date + 5
    df_temp = df[(df['사용일자'] >= start_date) & (df['사용일자'] < end_date)].copy()
    df_temp = df_temp.groupby(cols)[target].agg('sum').reset_index()
    df_list.append(df_temp)
df_1 = pd.concat(df_list, axis=0)
# df_1['사용일자'] = 202001

df_1 = df_1.groupby(cols)[target].agg('sum').reset_index()

In [60]:
df_1.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200106,1호선,동대문,13409,14525
1,20200106,1호선,동묘앞,7107,7716
2,20200106,1호선,서울역,58509,54106
3,20200106,1호선,시청,27431,27926
4,20200106,1호선,신설동,16605,16430


In [61]:
df_1.shape

(11810, 5)

In [62]:
# 수동
df1 = df[(df.사용일자 >= 20200101)&(df.사용일자 < 20200104)].copy()
df1.groupby(cols)[target].agg('sum').reset_index()
df1.tail()

,사용일자,노선명,역명,승차총승객수,하차총승객수
1769,20200103,우이신설선,북한산우이,2477,2140
1770,20200103,우이신설선,솔밭공원,2550,2489
1771,20200103,우이신설선,4.19민주묘지,2924,2752
1772,20200103,우이신설선,가오리,4409,4168
1773,20200103,우이신설선,화계,3557,3288


In [63]:
# 2월
cols = list(df.columns)[:3]
target = list(df.columns)[3:]
df_list = []
# 반복문에 해당되지 않는 범위는 수동으로 구해서 
for i in range(3, 25, 7): # 사용할때 여기 수정 해당 월에 맞게   여긴 1월2일이 월요일이라 2일 부터 시작 30,31일 데이터는 추가로 더해야함
    start_date = 20200200 + i   # 여기도 조건에 맞게 수정
    end_date = start_date + 5
    df_temp = df[(df['사용일자'] >= start_date) & (df['사용일자'] < end_date)].copy()
    df_temp = df_temp.groupby(cols)[target].agg('sum').reset_index()
    df_list.append(df_temp)
df_2 = pd.concat(df_list, axis=0)
# df_1['사용일자'] = 202001

df_2 = df_2.groupby(cols)[target].agg('sum').reset_index()
df_2.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200203,1호선,동대문,11981,12001
1,20200203,1호선,동묘앞,7572,7863
2,20200203,1호선,서울역,48307,46039
3,20200203,1호선,시청,25019,25472
4,20200203,1호선,신설동,15527,15344


In [64]:
# 이렇게 12개월치 구한거
# df_res_list = [df_1,df_2,df_3 ....df_12] 리스트 안에 넣음
# df_res = pd.concat(df_res_list, axis=0) 합침
# 이거에 이제 역명 '(' 지우고 하는 작업하고
# 호선별로 나눈다음에
# line_merge 코드 짜둔거 사용해서 합치면 됨

- 1~12월까지 구한 뒤 concat

In [65]:
df_1 = pd.concat([df_1, df1], axis=0)
df_1.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200106,1호선,동대문,13409,14525
1,20200106,1호선,동묘앞,7107,7716
2,20200106,1호선,서울역,58509,54106
3,20200106,1호선,시청,27431,27926
4,20200106,1호선,신설동,16605,16430


In [66]:
df_1.to_csv('./평일.csv', index=False)

In [67]:
df = pd.read_csv('./평일.csv')

In [68]:
# 1~12월 다 합치고 불러와서 아래 실행
# 호선명 리스트로 추출해서 사용
lines = df.노선명.unique().tolist()
df_dict = {line: df[df['노선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    # frame = df[df['노선명']==line].copy()
    frame['총 승차인원'] = frame.loc[:,['승차총승객수']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['하차총승객수']].sum(axis=1)

    frame.to_csv(f'./{line}.csv',index=False,encoding='utf-8')

In [69]:
# 저장 위치 고치기
line_info = [
    (['./1호선.csv', './경부선.csv', './경원선.csv', './경인선.csv', './장항선.csv'], '1호선'),
    (['./2호선.csv'], '2호선'),
    (['./3호선.csv', './일산선.csv'], '3호선'),
    (['./4호선.csv', './과천선.csv', './안산선.csv'], '4호선'),
    (['./5호선.csv'], '5호선'),
    (['./6호선.csv'], '6호선'),
    (['./7호선.csv'], '7호선'),
    (['./8호선.csv'], '8호선'),
    (['./9호선.csv', './9호선2단계.csv', './9호선2~3단계.csv'], '9호선'),
    (['./수인선.csv', './분당선.csv'], '수인분당선'),
    (['./경의선.csv', './중앙선.csv'], '경의중앙선')

]

for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    res.to_csv(f'./{line_name}.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: './9호선2단계.csv'